In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent)) 
from tool_monkey import MonkeyObserver, with_monkey, single_timeout, logger, setup_default_logging

In [ ]:

setup_default_logging(level=10)

Example 1: Using `bind_tools()`

In [9]:
def bind_tools_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_weather_tool 
    from langchain_core.tools import tool
    observer=MonkeyObserver()
    scenario=single_timeout(seconds=3.0)
    wrapped_base_weather_tool=with_monkey(scenario, observer)(base_weather_tool)
    @tool
    def get_weather(location: str, units: str = "celcius"):
        """Get the current weather for a given location.
            Args:
                location (str): The location to get the weather for.
                units (str): The units to return the weather in. Either 'celsius' or 'fahrenheit'.
            Returns:
                str: The current weather in the given location."""
        return wrapped_base_weather_tool(location, units)
    messages = [
    {"role": "system", "content": "You are a helpful assistant with access to a tool to get weather information."},
    {"role": "user", "content": "What's the weather in Boston?"}]
    llm_with_tool = llm.bind_tools([get_weather])
    # invoke the model with the messages
    ai_msg = llm_with_tool.invoke(messages)
    logger.debug(f"Full LLM Response: {ai_msg}")
    messages.append(ai_msg)
    logger.debug(f"Have tool calls? {'Yes' if ai_msg.tool_calls else 'No'}")
    try:
        for tool_call in ai_msg.tool_calls:
            logger.debug(
                f"Invoking tool: {tool_call.get("name")} with args {tool_call.get('args')}")
            tool_result = get_weather.invoke(tool_call)
            logger.debug(f"Tool result: {tool_result}")
            messages.append(tool_result)
        final_response = llm_with_tool.invoke(messages)
        logger.debug(f"Final LLM Response: {final_response.text}")
    except Exception:
        pass
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

bind_tools_example()

DEBUG: Full LLM Response: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 134, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CwsTgELuVSi3lUI8XSO1DR7G2teQJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019baddc-8f9b-7951-84f5-91fb08811a42-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1uRToUYDAvOhDMrhWnqkXMlv', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 134, 'output_tokens': 15, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
DEBUG: Have tool cal


OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 1
  Success Rate: 0.0%
  Failures: 1
  Total Retries: 0
  Avg Latency: 3001.5ms


Example 2: Using `create_agent`

In [13]:
def create_agent_example():
    from langchain.agents import create_agent
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_weather_tool 
    from langchain_core.tools import tool
    observer=MonkeyObserver()
    scenario=single_timeout(seconds=3.0)
    wrapped_base_weather_tool=with_monkey(scenario, observer)(base_weather_tool)
    @tool
    def get_weather(location: str, units: str = "celcius"):
        """Get the current weather for a given location.
            Args:
                location (str): The location to get the weather for.
                units (str): The units to return the weather in. Either 'celsius' or 'fahrenheit'.
            Returns:
                str: The current weather in the given location."""
        return wrapped_base_weather_tool(location, units)
    messages = [
    {"role": "system", "content": "You are a helpful assistant with access to a tool to get weather information."},
    {"role": "user", "content": "What's the weather in Boston?"}]
    agent=create_agent(model=llm, tools=[get_weather])
    try:
        response=agent.invoke(input={"messages":messages})
        final_message=response["messages"][-1]
        print(f"Final response: {final_message.content}")
    except Exception:
        pass
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

create_agent_example()

    

INFO: Simulating timeout for 3.0 seconds



OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 1
  Success Rate: 0.0%
  Failures: 1
  Total Retries: 0
  Avg Latency: 3002.0ms


Example 3: Using a retry strategy (with `tenacity`)

In [18]:
def timeout_retry_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_weather_tool 
    from langchain_core.tools import tool
    from tenacity import retry, stop_after_attempt, wait_exponential_jitter
    observer=MonkeyObserver()
    scenario=single_timeout(seconds=3.0)

    wrapped_weather_tool=with_monkey(scenario, observer)(base_weather_tool)
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential_jitter(initial=0.2, max=2.0),
        retry_error_callback=lambda retry_state: "Weather data unavailable due to timeout",
    )
    def weather_tool_with_retry(location:str, units:str="celcius"):
        return wrapped_weather_tool(location, units)
    

    @tool
    def get_weather(location: str, units: str = "celcius"):
        """Get the current weather for a given location.
            Args:
                location (str): The location to get the weather for.
                units (str): The units to return the weather in. Either 'celsius' or 'fahrenheit'.
            Returns:
                str: The current weather in the given location."""
        return weather_tool_with_retry(location, units)
    
    messages = [
    {"role": "system", "content": "You are a helpful assistant with access to a tool to get weather information."},
    {"role": "user", "content": "What's the weather in Boston?"}]
    llm_with_tool = llm.bind_tools([get_weather])
    # invoke the model with the messages
    ai_msg = llm_with_tool.invoke(messages)
    logger.debug(f"Full LLM Response: {ai_msg}")
    messages.append(ai_msg)
    logger.debug(f"Have tool calls? {'Yes' if ai_msg.tool_calls else 'No'}")
    try:
        for tool_call in ai_msg.tool_calls:
            logger.debug(
                f"Invoking tool: {tool_call.get("name")} with args {tool_call.get('args')}")
            tool_result = get_weather.invoke(tool_call)
            logger.debug(f"Tool result: {tool_result}")
            messages.append(tool_result)
        final_response = llm_with_tool.invoke(messages)
        logger.debug(f"Final LLM Response: {final_response.text}")
    except Exception:
        pass
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

timeout_retry_example()

DEBUG: Full LLM Response: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 134, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CwtfQsbgs5kgZCOW6Lge6IA2CIdO7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bae22-5207-7b21-a139-3e18278483ab-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_wt8NfTXmIWZYBxbiHXaMH6VZ', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 134, 'output_tokens': 15, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
DEBUG: Have tool cal


OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 2
  Success Rate: 50.0%
  Failures: 1
  Total Retries: 0
  Avg Latency: 1500.5ms
